# Code to run ElasticNet regression to predict TMI from rs-fMRI data and covariates
Code by Ilkka Suuronen; adapted by Isabella L.C. Mariani Wigley and Aurora Berto for PONS project (06 / 2025)

ilksuu@utu.fi; ilmawi@utu.fi; aurber@utu.fi

In [1]:
# import libraries and check versions

import numpy as np
import pandas as pd
import sklearn
import matplotlib
import scipy
import os
import sys

from itertools import product

from matplotlib import pyplot as plt

from sklearn.linear_model import ElasticNetCV

import warnings
warnings.filterwarnings("ignore")

print(sys.version)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(matplotlib.__version__)
print(scipy.__version__)

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ]
1.26.4
2.3.0
1.7.0
3.10.3
1.15.3


In [2]:
# particular sklearn modules to use in ML analyses

from sklearn.model_selection import cross_validate, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import clone

In [3]:
# path to the directory containing the data

tabularData_root = r"/Users/auroraberto/Desktop/24-25/MSc_thesis/main_article/createDataset"
# tabularData_root = r"/scratch/project_2006897/bertoaur/7_mainArticle_Dataset"

In [4]:
# load the dataset
data_path = os.path.join(tabularData_root, "main_dataset_rsfMRI_FCH_LEiDA_demo.csv") 
data = pd.read_csv(data_path)
data

,src_subject_id,eventname,anthro_1_height_in,anthroweight1lb,demo_sex_v2,race_ethnicity,mri_info_visitid,mri_info_deviceserialnumber,interview_age,rel_family_id,...,Harmonics_energy108,Harmonics_energy109,Harmonics_energy110,Harmonics_energy111,Harmonics_energy112,Harmonics_energy113,pubertal_developmental_scale,triponderal_mass_index,birth_weight_g,batch_id
0,NDAR_INV003RTV85,baseline_year_1_arm_1,56.5,93.0,2.0,1.0,S042_INV003RTV85_baseline,HASH96a0c182,131.0,8781,...,1.491364e+07,2.719445e+07,2.713050e+07,8.966750e+07,7.367208e+07,8.757473e+06,3.0,14.272570,3175.14659,S042
1,NDAR_INV00BD7VDC,baseline_year_1_arm_1,57.5,76.8,1.0,1.0,S090_INV00BD7VDC_baseline,HASH65b39280,112.0,3810,...,1.432104e+07,2.813274e+07,2.206341e+07,5.290427e+07,5.142680e+07,1.302604e+07,2.0,11.182072,3628.73896,S090
2,NDAR_INV00CY2MDM,baseline_year_1_arm_1,56.5,91.5,1.0,1.0,S021_INV00CY2MDM_baseline,HASHd422be27,130.0,5355,...,7.229910e+06,2.426002e+07,2.155624e+07,6.341999e+07,6.357962e+07,1.216644e+07,2.0,14.042368,2721.55422,S021
3,NDAR_INV00HEV6HB,baseline_year_1_arm_1,57.3,70.8,1.0,2.0,S012_INV00HEV6HB_baseline,HASHe4f6957a,124.0,2257,...,1.326408e+07,2.637955e+07,2.615203e+07,1.033358e+08,6.688947e+07,7.337214e+06,2.0,10.416792,2721.55422,S012
4,NDAR_INV00LH735Y,baseline_year_1_arm_1,52.0,80.0,1.0,3.0,S011_INV00LH735Y_baseline,HASH5b0cf1bb,109.0,6069,...,1.541493e+07,6.418400e+07,2.960504e+07,6.625194e+07,4.544425e+07,1.200974e+07,1.0,15.748694,3175.14659,S011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,NDAR_INVZZ6ZJ2KY,baseline_year_1_arm_1,57.0,111.0,2.0,1.0,S042_INVZZ6ZJ2KY_baseline,HASH96a0c182,124.0,9345,...,1.134298e+07,3.480763e+07,1.708301e+07,5.452299e+07,6.199466e+07,5.114478e+06,3.0,16.590635,3628.73896,S042
4449,NDAR_INVZZ81LEEV,baseline_year_1_arm_1,53.5,57.8,1.0,2.0,S076_INVZZ81LEEV_baseline,HASH03db707f,108.0,8433,...,1.442863e+07,2.144304e+07,2.867833e+07,4.724747e+07,3.819962e+07,1.254573e+07,2.0,10.447950,2721.55422,S076
4450,NDAR_INVZZJ3A7BK,baseline_year_1_arm_1,59.0,137.0,2.0,1.0,S042_INVZZJ3A7BK_baseline,HASH96a0c182,122.0,9346,...,1.261651e+07,5.359243e+07,1.434609e+07,7.270545e+07,3.830621e+07,1.226686e+07,3.0,18.464142,3628.73896,S042
4451,NDAR_INVZZLZCKAY,baseline_year_1_arm_1,59.5,123.0,2.0,1.0,S042_INVZZLZCKAY_baseline,HASH96a0c182,110.0,9347,...,9.107507e+06,4.535081e+07,1.665423e+07,3.659193e+07,5.079769e+07,1.255092e+07,3.0,16.162882,3175.14659,S042


In [5]:
# define columns for different analysis assignments

# covariates
num_cov_cols = ["interview_age", "pubertal_developmental_scale"] # numeric covariates
cat_cov_cols = ["demo_sex_v2", "race_ethnicity", "mri_info_deviceserialnumber"] # categorical covariates

In [6]:
# rs-fMRI features
rsfmri_pattern = "^(rsfmri)" # define a pattern to catch all brain imaging features
rsfmri_cols = list(data.filter(regex=rsfmri_pattern).columns) # turn pattern to a list of column names

In [ ]:
import re

# LEiDA features
leida_pattern = "^(P_k|LT_k|TR_K)"
leida_cols = list(data.filter(regex=leida_pattern).columns)

## Discard not matching networks

# probabilities and lifetimes
leida_to_remove = ["P_k5c5","P_k10c5", "P_k12c5", "P_k15c7", "P_k16c9", "P_k18c11", "P_k19c11", "P_k15c12",
                   "P_k20c12", "P_k14c13", "P_k17c13", "P_k20c14", "P_k15c15", "P_k20c17", "P_k19c19", 
                   "P_k20c20", "LT_k5c5", "LT_k10c5", "LT_k12c5", "LT_k15c7", "LT_k16c9", "LT_k18c11", "LT_k19c11",
                   "LT_k15c12", "LT_k20c12", "LT_k14c13", "LT_k17c13", "LT_k20c14", "LT_k15c15", "LT_k20c17",
                   "LT_k19c19", "LT_k20c20"
                    ]

# from probabilities and lifetimes extract couples to discard transitions
k_c_pairs = []
for entry in leida_to_remove:
    match = re.search(r'k(\d+)c(\d+)', entry)
    if match:
        k = int(match.group(1))
        c = int(match.group(2))
        k_c_pairs.append((k, c))

# find and remove transitions
transitions_to_remove = []
for col in leida_cols:
    match = re.match(r'TR_K(\d+)_C(\d+)x(\d+)', col)
    if match:
        k = int(match.group(1))
        c1 = int(match.group(2))
        c2 = int(match.group(3))
        if any(k == k0 and (c1 == c0 or c2 == c0) for (k0, c0) in k_c_pairs):
            transitions_to_remove.append(col)

# merge all columns to remove
all_to_remove = list(set(leida_to_remove + transitions_to_remove))

# Keep only columns to retain
leida_cols = [col for col in leida_cols if col not in all_to_remove]


## Select only LEiDA metrics for K=20
leida_pattern_cut = "^(P_k20|LT_k20|TR_K20)"
leida_cols_cut = list(data.filter(regex=leida_pattern_cut).columns)

In [ ]:
# HARMONICS features
harmonics_pattern = "^(Harmonics)"
harmonics_cols = list(data.filter(regex=harmonics_pattern).columns)

# Keep only the first 20 harmonics
harmonics_cols = [
    col for col in harmonics_cols
    if (
        (re.search(r'Harmonics_power(\d+)', col) and 1 <= int(re.search(r'Harmonics_power(\d+)', col).group(1)) <= 20)
        or
        (re.search(r'Harmonics_energy(\d+)', col) and 1 <= int(re.search(r'Harmonics_energy(\d+)', col).group(1)) <= 20)
    )
]

In [9]:
# All imaging features
img_cols = rsfmri_cols + leida_cols + harmonics_cols
# img_cols = rsfmri_cols + leida_cols_cut + harmonics_cols

# all numeric features
num_cols = num_cov_cols + img_cols

In [10]:
# define pipelines for numeric and categorical columns
num_pipe = make_pipeline(SimpleImputer(strategy="median")) # median instead of mean (more robust)
cat_pipe = make_pipeline(SimpleImputer(strategy="most_frequent"),
                         OneHotEncoder(handle_unknown='ignore', sparse_output=False))

# define base elasticnet model with hyperparameter grid
base_model = ElasticNetCV(l1_ratio=[.01, .025, .05, .1, .5, .7, .9, 1], 
                        alphas=[2**x for x in range(-8, 8)], max_iter = 100, # max_iter = 10000,
                        n_jobs=-1, cv=KFold(10, shuffle=True, random_state=33), verbose=0)

In [11]:
## MODEL WITH COVARIATES
# define a transformer that branches for different column types
ct_cov = ColumnTransformer([("num_transformer", num_pipe, num_cov_cols),
                        ("cat_transformer", cat_pipe, cat_cov_cols)], verbose_feature_names_out=False)
# define imaging columns elasticnet prediction pipeline
model_cov = Pipeline([("preprocessing", ct_cov), ("scaler", StandardScaler()),
                         ("predictor", base_model)]).set_output(transform="pandas")

In [12]:
## MODEL WITH RS-fMRI DATA
# define a transformer that branches for different column types
ct_rsfmri = ColumnTransformer([("num_transformer", num_pipe, num_cov_cols + rsfmri_cols),
                        ("cat_transformer", cat_pipe, cat_cov_cols)], verbose_feature_names_out=False)
# define imaging columns elasticnet prediction pipeline
model_rsfmri = Pipeline([("preprocessing", ct_rsfmri), ("scaler", StandardScaler()),
                         ("predictor", base_model)]).set_output(transform="pandas")

In [13]:
## MODEL WITH LEiDA DATA
# define a transformer that branches for different column types
ct_leida = ColumnTransformer([("num_transformer", num_pipe, num_cov_cols + leida_cols), # to change to leida_cols/leida_cols_cut accordingly
                        ("cat_transformer", cat_pipe, cat_cov_cols)], verbose_feature_names_out=False)
# define imaging columns elasticnet prediction pipeline
model_leida = Pipeline([("preprocessing", ct_leida), ("scaler", StandardScaler()),
                         ("predictor", base_model)]).set_output(transform="pandas")

In [14]:
## MODEL WITH HARMONICS DATA
# define a transformer that branches for different column types
ct_harmonics = ColumnTransformer([("num_transformer", num_pipe, num_cov_cols + harmonics_cols),
                        ("cat_transformer", cat_pipe, cat_cov_cols)], verbose_feature_names_out=False)
# define imaging columns elasticnet prediction pipeline
model_harmonics = Pipeline([("preprocessing", ct_harmonics), ("scaler", StandardScaler()),
                         ("predictor", base_model)]).set_output(transform="pandas")

In [15]:
## MODEL WITH ALLA DATA
# define a transformer that branches for different column types
ct_all = ColumnTransformer([("num_transformer", num_pipe, num_cov_cols + img_cols),
                        ("cat_transformer", cat_pipe, cat_cov_cols)], verbose_feature_names_out=False)
# define all columns elasticnet prediction pipeline
model_all = Pipeline([("preprocessing", ct_all), ("scaler", StandardScaler()),
                         ("predictor", base_model)]).set_output(transform="pandas")

In [16]:
# define X, y for different analysis settings
X_cov = data.loc[:, num_cov_cols + cat_cov_cols]
X_rsfmri = data.loc[:, rsfmri_cols + num_cov_cols + cat_cov_cols]
X_leida = data.loc[:, leida_cols + num_cov_cols + cat_cov_cols] # leida_cols/leida_cols_cut
X_harmonics = data.loc[:, harmonics_cols + num_cov_cols + cat_cov_cols]

X_all = data.loc[:, img_cols + num_cov_cols + cat_cov_cols]

y = data.loc[:, "triponderal_mass_index"]

In [18]:
# define a function to display CV results in a pandas dataframe for easier reading

def make_result_df(cv_res):
        
    # define metrics
    metrics = ["r2", "neg_mean_absolute_error"]
    # define statistical functions
    statfuncs = [np.mean, np.std]
    # define cross-validation fold names
    folds = ["train", "test"]
    
    # get names for statistical functions
    statfunc_names = [s.__name__ for s in statfuncs]
    # make column names of type "train_mean" etc. using product
    table_cols = [p[0] + "_" + p[1] for p in list(product(folds, statfunc_names))]
    # init. empty dataframe with cols and metrics as index
    result_df = pd.DataFrame(columns=table_cols, index=metrics)
    # nested loop to fill a dataframe
    for f in folds:
        for m in result_df.index:
            # k for key to get cross-validation results from cv result dictionary
            k = f + "_" + m
            for s in statfuncs:
                # compute mean/std for train/test result
                stat = s(cv_res[k])
                # fill into previously empty df
                result_df.loc[m][f + "_" + s.__name__] = stat
                
    return result_df

In [37]:
# only covariates (base model), elastic net

res_cov = cross_validate(model_cov, X=X_cov, y=y, scoring=["r2", "neg_mean_absolute_error"],
                        cv=KFold(5, shuffle=True, random_state=44),
                        return_estimator=True, return_train_score=True, verbose=0)

make_result_df(res_cov)

,train_mean,train_std,test_mean,test_std
r2,0.108565,0.003702,0.089535,0.011771
neg_mean_absolute_error,-1.75257,0.012068,-1.767901,0.029708


In [38]:
# rs-fMRI features and covariates, elasticnet

res_rsfmri = cross_validate(model_rsfmri, X=X_rsfmri, y=y, scoring=["r2", "neg_mean_absolute_error"],
                        cv=KFold(5, shuffle=True, random_state=44),
                        return_estimator=True, return_train_score=True, verbose=0)

make_result_df(res_rsfmri)

,train_mean,train_std,test_mean,test_std
r2,0.172505,0.014648,0.113199,0.014266
neg_mean_absolute_error,-1.688771,0.021918,-1.741271,0.023257


In [19]:
# LEiDA features and covariates, elasticnet

res_leida = cross_validate(model_leida, X=X_leida, y=y, scoring=["r2", "neg_mean_absolute_error"],
                        cv=KFold(5, shuffle=True, random_state=44),
                        return_estimator=True, return_train_score=True, verbose=0)

make_result_df(res_leida)

,train_mean,train_std,test_mean,test_std
r2,0.101409,0.004404,0.073528,0.005867
neg_mean_absolute_error,-1.772529,0.010707,-1.789932,0.027332


In [40]:
# harmonics features and covariates, elasticnet

res_harmonics = cross_validate(model_harmonics, X=X_harmonics, y=y, scoring=["r2", "neg_mean_absolute_error"],
                        cv=KFold(5, shuffle=True, random_state=44),
                        return_estimator=True, return_train_score=True, verbose=0)

make_result_df(res_harmonics)

,train_mean,train_std,test_mean,test_std
r2,0.114164,0.007158,0.084143,0.008608
neg_mean_absolute_error,-1.7489,0.017635,-1.774001,0.028561


In [20]:
# imaging features (rs-fMRI, LEiDA and harmonics) and covariates, elasticnet

res_all = cross_validate(model_all, X=X_all, y=y, scoring=["r2", "neg_mean_absolute_error"],
                        cv=KFold(5, shuffle=True, random_state=44),
                        return_estimator=True, return_train_score=True, verbose=0)

make_result_df(res_all)

,train_mean,train_std,test_mean,test_std
r2,0.154358,0.011574,0.091943,0.004926
neg_mean_absolute_error,-1.718021,0.019173,-1.767812,0.02891
